In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import StrMethodFormatter

import os
from os import path
from glob import glob
import sys
import csv
sys.path.insert(0, path.abspath('./'))

from src import workdir, parse_model_parameter_file
from src.emulator_BAND import EmulatorBAND
from src.emulator import Emulator


# Define functions to compare multiple GP emulators 

In [ ]:
def rms_abs_prediction_err(emu_pred,vali_true):
    rms_abs_pred_err = np.zeros(emu_pred.shape[1])
    for obsIdx in range(emu_pred.shape[1]):
        quantity = np.zeros(emu_pred.shape[1])
        for testpoint in range(emu_pred.shape[0]):
            quantity[obsIdx] += ((emu_pred[testpoint,obsIdx] - vali_true[testpoint,obsIdx]) / vali_true[testpoint,obsIdx])**2.
        rms_abs_pred_err[obsIdx] = np.sqrt(quantity[obsIdx] / emu_pred.shape[0])
    return rms_abs_pred_err

def how_honest_is_GP(emu_pred,emu_pred_err,vali_true):
    rms_quantity = np.zeros(emu_pred.shape[1])
    for obsIdx in range(emu_pred.shape[1]):
        quantity = np.zeros(emu_pred.shape[1])
        for testpoint in range(emu_pred.shape[0]):
            quantity[obsIdx] += ((emu_pred[testpoint,obsIdx] - vali_true[testpoint,obsIdx]) / emu_pred_err[testpoint,obsIdx])**2.
        rms_quantity[obsIdx] = np.sqrt(quantity[obsIdx] / emu_pred.shape[0])
    return rms_quantity

def train_multiple_emulators(training_set, model_par, number_test_points, logFlag, parameterTrafoPCAFlag):
    emu1 = EmulatorBAND(training_set, model_par, method='PCGP', logTrafo=logFlag, parameterTrafoPCA=parameterTrafoPCAFlag)
    emu2 = EmulatorBAND(training_set, model_par, method='PCSK', logTrafo=logFlag, parameterTrafoPCA=parameterTrafoPCAFlag)
    emu3 = Emulator(training_set, model_par, npc = 4, logTrafo=logFlag, parameterTrafoPCA=parameterTrafoPCAFlag)

    output_emu1 = emu1.testEmulatorErrors(number_test_points=number_test_points)
    emu_pred_1 = output_emu1[0]
    emu_pred_err_1 = output_emu1[1]
    vali_data_1 = output_emu1[2]
    vali_data_err_1 = output_emu1[3]

    output_emu2 = emu2.testEmulatorErrors(number_test_points=number_test_points)
    emu_pred_2 = output_emu2[0]
    emu_pred_err_2 = output_emu2[1]
    vali_data_2 = output_emu2[2]
    vali_data_err_2 = output_emu2[3]

    output_emu3 = emu3.testEmulatorErrors(nTestPoints=number_test_points)
    emu_pred_3 = output_emu3[0]
    emu_pred_err_3 = output_emu3[1]
    vali_data_3 = output_emu3[2]
    vali_data_err_3 = output_emu3[3]

    nObs = vali_data_1.shape[1]  # Assuming all datasets have the same number of observables

    rms_abs_pred_err1 = rms_abs_prediction_err(emu_pred_1,vali_data_1)
    rms_abs_pred_err2 = rms_abs_prediction_err(emu_pred_2,vali_data_2)
    rms_abs_pred_err3 = rms_abs_prediction_err(emu_pred_3,vali_data_3)
    honesty_1 = how_honest_is_GP(emu_pred_1,emu_pred_err_1,vali_data_1)
    honesty_2 = how_honest_is_GP(emu_pred_2,emu_pred_err_2,vali_data_2)
    honesty_3 = how_honest_is_GP(emu_pred_3,emu_pred_err_3,vali_data_3)

    return (rms_abs_pred_err1,rms_abs_pred_err2,rms_abs_pred_err3), (honesty_1,honesty_2,honesty_3)


Write functions for the output of the different GP's

In [ ]:
def write_output_to_csv_uncertainties(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow([row])

def train_multiple_emulators_and_write_to_csv(training_set, model_par, number_test_points, output_file, logFlag, parameterPCAFlag):
    (rms_abs_pred_err1, rms_abs_pred_err2, rms_abs_pred_err3), (honesty_1, honesty_2, honesty_3) = train_multiple_emulators(training_set, model_par, number_test_points, logFlag, parameterPCAFlag)

    # Write rms_abs_pred_err1, rms_abs_pred_err2, rms_abs_pred_err3 to CSV
    write_output_to_csv_uncertainties(output_file + f'_{number_test_points}_pred_err_obs_1.dat', rms_abs_pred_err1)
    write_output_to_csv_uncertainties(output_file + f'_{number_test_points}_pred_err_obs_2.dat', rms_abs_pred_err2)
    write_output_to_csv_uncertainties(output_file + f'_{number_test_points}_pred_err_obs_3.dat', rms_abs_pred_err3)

    # Write honesty_1, honesty_2, honesty_3 to CSV
    write_output_to_csv_uncertainties(output_file + f'_{number_test_points}_GP_honesty_obs_1.dat', honesty_1)
    write_output_to_csv_uncertainties(output_file + f'_{number_test_points}_GP_honesty_obs_2.dat', honesty_2)
    write_output_to_csv_uncertainties(output_file + f'_{number_test_points}_GP_honesty_obs_3.dat', honesty_3)

Read functions for the different file types

In [ ]:
def read_emulator_file_errors(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            data.append(float(line.strip()))
    return data

def read_multiple_emulator_errors_files(number_test_points_list,foldername,filename):
    data_list1 = []
    data_list2 = []
    data_list3 = []
    for i in number_test_points_list:
        filename1 = f"./{foldername}/{filename}_{i}_pred_err_obs_1.dat"
        data1 = read_emulator_file_errors(filename1)
        data_list1.append(data1)

        filename2 = f"./{foldername}/{filename}_{i}_pred_err_obs_2.dat"
        data2 = read_emulator_file_errors(filename2)
        data_list2.append(data2)

        filename3 = f"./{foldername}/{filename}_{i}_pred_err_obs_3.dat"
        data3 = read_emulator_file_errors(filename3)
        data_list3.append(data3)
    
    data_list4 = []
    data_list5 = []
    data_list6 = []
    for i in number_test_points_list:
        filename4 = f"./{foldername}/{filename}_{i}_GP_honesty_obs_1.dat"
        data4 = read_emulator_file_errors(filename4)
        data_list4.append(data4)

        filename5 = f"./{foldername}/{filename}_{i}_GP_honesty_obs_2.dat"
        data5 = read_emulator_file_errors(filename5)
        data_list5.append(data5)

        filename6 = f"./{foldername}/{filename}_{i}_GP_honesty_obs_3.dat"
        data6 = read_emulator_file_errors(filename6)
        data_list6.append(data6)
    return (data_list1,data_list2,data_list3), (data_list4,data_list5,data_list6)


Generate the data files for a different number of test points

In [ ]:
model_par = "../data/modelDesign_3DMCGlauber.txt"
path_output = './emulator_output_PCA/'
data_path = "../data/"
if not os.path.exists(path_output):
    os.makedirs(path_output)

training_set_list_dNdy = ["AuAu7.7_dNdy.pkl","AuAu19p6_dNdy.pkl","AuAu200_dNdy.pkl","AuAu200_PHOBOSdNdeta.pkl","AuAu19p6_PHOBOSdNdeta.pkl"]
output_file_list_dNdy = ["7p7_dNdy_emu_out","19p6_dNdy_emu_out","200_dNdy_emu_out","200PHOBOS_dNdeta_emu_out","19p6PHOBOS_dNdeta_emu_out"]
output_file_list_LOGdNdy = ["7p7_LOGdNdy_emu_out","19p6_LOGdNdy_emu_out","200_LOGdNdy_emu_out","200PHOBOS_LOGdNdeta_emu_out","19p6PHOBOS_LOGdNdeta_emu_out"]

training_set_list_pTvn = ["AuAu7.7_pTvn.pkl","AuAu19p6_pTvn.pkl","AuAu200_pTvn.pkl", "AuAu200_PHOBOSv2eta.pkl"]
output_file_list_pTvn = ["7p7_pTvn_emu_out","19p6_pTvn_emu_out","200_pTvn_emu_out","200PHOBOS_vn_emu_out"]

In [ ]:
for i in [700]:
    for tr_set in range(len(training_set_list_dNdy)):
        train_multiple_emulators_and_write_to_csv(data_path + training_set_list_dNdy[tr_set], model_par, i, path_output + output_file_list_dNdy[tr_set], False, True)

In [ ]:
for i in [700]:
    for tr_set in range(len(training_set_list_dNdy)):
        train_multiple_emulators_and_write_to_csv(data_path + training_set_list_dNdy[tr_set], model_par, i, path_output + output_file_list_LOGdNdy[tr_set], True, True)

In [ ]:
for i in [700]:
    for tr_set in range(len(training_set_list_pTvn)):
        train_multiple_emulators_and_write_to_csv(data_path + training_set_list_pTvn[tr_set], model_par, i, path_output + output_file_list_pTvn[tr_set], False, True)

Quick summary:
- The AuAu7.7_dNdy data set does not sort out any of the 1100 training points.
- The AuAu7.7_pTvn data set sorts out 40 of the 1100 training points due to large statistical errors.
- The AuAu19p6_dNdy data set does not sort out any of the 1100 training points.
- The AuAu19p6_pTvn data set sorts out 5 of the 1100 training points due to large statistical errors.
- The AuAu19p6_PHOBOS_dNdeta data set does not sort out any of the 1100 points.
- The AuAu200_dNdy data set does not sort out any of the 1100 training points.
- The AuAu200_pTvn data set sorts out 46 of the 1100 training points due to large statistical errors.
- The AuAu200_PHOBOS_dNdeta data set does not sort out any of the 1100 training points.
- The AuAu200_PHOBOSv2eta data set does not sort out any of the 1100 training points.

Final plot functions

In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out","7p7_pTvn_emu_out","19p6_dNdy_emu_out","19p6_pTvn_emu_out","200_dNdy_emu_out","200_pTvn_emu_out","19p6PHOBOS_dNdeta_emu_out","200PHOBOS_dNdeta_emu_out","200PHOBOS_vn_emu_out"]
foldername = 'emulator_outputs_only_LHC/emulator_output'

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([30],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

colors = ['r','b','g','magenta','teal','orange','purple','grey','seagreen']
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{7.7\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'$\langle p_{\rm T}\rangle|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{200\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          r'$v_2^{\rm ch}(\eta)|^{200\;\mathrm{GeV}}$'
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        axs[0].scatter(idx_observable, np.abs(err1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
        axs[0].scatter(idx_observable, np.abs(err2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
        axs[0].scatter(idx_observable, np.abs(err3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(err1_list)):  # Iterate over the remaining data sets
    for obs in range(len(err1_list[dataset][-1])):
        axs[1].scatter(idx_observable, np.abs(err1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
        axs[1].scatter(idx_observable, np.abs(err2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
        axs[1].scatter(idx_observable, np.abs(err3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

legend_elements = [
    plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
    plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
    plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None')
]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
plt.savefig("err_full_training_set_LHCpoints_noPCA.pdf")

# honesty plot
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon1_list[dataset][-1])):
        axs[0].scatter(idx_observable, np.log(np.abs(hon1_list[dataset][-1][obs])), color=colors[0], marker='o', s=10)
        axs[0].scatter(idx_observable, np.log(np.abs(hon2_list[dataset][-1][obs])), color=colors[1], marker='s', s=10)
        axs[0].scatter(idx_observable, np.log(np.abs(hon3_list[dataset][-1][obs])), color=colors[2], marker='^', s=10)
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(hon1_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon1_list[dataset][-1])):
        axs[1].scatter(idx_observable, np.log(np.abs(hon1_list[dataset][-1][obs])), color=colors[0], marker='o', s=10)
        axs[1].scatter(idx_observable, np.log(np.abs(hon2_list[dataset][-1][obs])), color=colors[1], marker='s', s=10)
        axs[1].scatter(idx_observable, np.log(np.abs(hon3_list[dataset][-1][obs])), color=colors[2], marker='^', s=10)
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

legend_elements = [
    plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
    plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
    plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None')
]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.02, borderaxespad=0.1)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-1, 1])
axs[1].set_ylim([-1, 1])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
plt.savefig("hon_full_training_set_LHCpoints_noPCA.pdf")


In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out","19p6_dNdy_emu_out","200_dNdy_emu_out","19p6PHOBOS_dNdeta_emu_out","200PHOBOS_dNdeta_emu_out"]
filename_prefix_list_LOG = ["7p7_LOGdNdy_emu_out","19p6_LOGdNdy_emu_out","200_LOGdNdy_emu_out","19p6PHOBOS_LOGdNdeta_emu_out","200PHOBOS_LOGdNdeta_emu_out"]
foldername = 'emulator_outputs_only_LHC/emulator_output'

emulator = 0

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([30],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

err1_list_PCA = []
err2_list_PCA = []
err3_list_PCA = []
hon1_list_PCA = []
hon2_list_PCA = []
hon3_list_PCA = []
for file in filename_prefix_list_LOG:
    err, hon = read_multiple_emulator_errors_files([30],foldername,file)
    err1_list_PCA.append(err[0])
    err2_list_PCA.append(err[1])
    err3_list_PCA.append(err[2])
    hon1_list_PCA.append(hon[0])
    hon2_list_PCA.append(hon[1])
    hon3_list_PCA.append(hon[2])

light_red = (1.0, 0.6, 0.6)  # Light red
light_blue = (0.6, 0.6, 1.0)  # Light blue
light_green = (0.6, 0.9, 0.6)  # Light green
colors = ['r','b','g',light_red,light_blue,light_green]
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err1_list[:3])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[:3]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(3, len(err1_list)):  # Iterate over the remaining data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[3:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:3], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[3:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP log', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK log', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP log', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
if emulator == 0:
    plt.savefig("err_full_training_set_LOG_PCGP_noPCA.pdf")
elif emulator == 1:
    plt.savefig("err_full_training_set_LOG_PCSK_noPCA.pdf")
elif emulator == 2:
    plt.savefig("err_full_training_set_LOG_Scikit_noPCA.pdf")

# honesty plot
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon1_list[:3])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[:3]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(3, len(hon1_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[3:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:3], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[3:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP log', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK log', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP log', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-1, 1])
axs[1].set_ylim([-1, 1])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
if emulator == 0:
    plt.savefig("hon_full_training_set_LOG_PCGP_noPCA.pdf")
elif emulator == 1:
    plt.savefig("hon_full_training_set_LOG_PCSK_noPCA.pdf")
elif emulator == 2:
    plt.savefig("hon_full_training_set_LOG_Scikit_noPCA.pdf")


Compare the emulators with and without the posterior points.

In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out","7p7_pTvn_emu_out","19p6_dNdy_emu_out","19p6_pTvn_emu_out","200_dNdy_emu_out","200_pTvn_emu_out","19p6PHOBOS_dNdeta_emu_out","200PHOBOS_dNdeta_emu_out","200PHOBOS_vn_emu_out"]
foldername = 'emulator_outputs_with_posterior/emulator_output'
foldername1 = 'emulator_outputs_with_posterior/emulator_output'

emulator = 0

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([100],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

err1_list_PCA = []
err2_list_PCA = []
err3_list_PCA = []
hon1_list_PCA = []
hon2_list_PCA = []
hon3_list_PCA = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([30],foldername1,file)
    err1_list_PCA.append(err[0])
    err2_list_PCA.append(err[1])
    err3_list_PCA.append(err[2])
    hon1_list_PCA.append(hon[0])
    hon2_list_PCA.append(hon[1])
    hon3_list_PCA.append(hon[2])

light_red = (1.0, 0.6, 0.6)  # Light red
light_blue = (0.6, 0.6, 1.0)  # Light blue
light_green = (0.6, 0.9, 0.6)  # Light green
colors = ['r','b','g',light_red,light_blue,light_green]
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{7.7\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'$\langle p_{\rm T}\rangle|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{200\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          r'$v_2^{\rm ch}(\eta)|^{200\;\mathrm{GeV}}$'
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(err1_list)):  # Iterate over the remaining data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP LHD', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP LHD + HPP', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK LHD', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK LHD + HPP', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP LHD', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP LHD + HPP', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
if emulator == 0:
    plt.savefig("err_LHC_vs_POST_PCGP_noPCA.pdf")
elif emulator == 1:
    plt.savefig("err_LHC_vs_POST_PCSK_noPCA.pdf")
elif emulator == 2:
    plt.savefig("err_LHC_vs_POST_Scikit_noPCA.pdf")

# honesty plot
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(hon1_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP LHD', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP LHD + HPP', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK LHD', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK LHD + HPP', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP LHD', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP LHD + HPP', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-2.5, 2.5])
axs[1].set_ylim([-2.5, 2.5])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
if emulator == 0:
    plt.savefig("hon_LHC_vs_POST_PCGP_noPCA.pdf")
elif emulator == 1:
    plt.savefig("hon_LHC_vs_POST_PCSK_noPCA.pdf")
elif emulator == 2:
    plt.savefig("hon_LHC_vs_POST_Scikit_noPCA.pdf")


Compute the average $\mathcal{E}$ and $\mathcal{H}$ for the emulators:

In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out","7p7_pTvn_emu_out","19p6_dNdy_emu_out","19p6_pTvn_emu_out","200_dNdy_emu_out","200_pTvn_emu_out","19p6PHOBOS_dNdeta_emu_out","200PHOBOS_dNdeta_emu_out","200PHOBOS_vn_emu_out"]
foldername = 'emulator_outputs_with_posterior/emulator_output'

emulator = 2

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([30],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

PCGP_E = 0.
PCSK_E = 0.
SCIKIT_E = 0.

PCGP_H = 0.
PCSK_H = 0.
SCIKIT_H = 0.

num_obs = 0
for dataset in range(len(err1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        num_obs += 1
        if emulator == 0:
            PCGP_E += err1_list[dataset][-1][obs]
            PCGP_H += np.log(hon1_list[dataset][-1][obs])
        elif emulator == 1:
            PCSK_E += err2_list[dataset][-1][obs]
            PCSK_H += np.log(hon2_list[dataset][-1][obs])
        elif emulator == 2:
            SCIKIT_E += err3_list[dataset][-1][obs]
            SCIKIT_H += np.log(hon3_list[dataset][-1][obs])

PCGP_E /= num_obs
PCSK_E /= num_obs
SCIKIT_E /= num_obs
PCGP_H /= num_obs
PCSK_H /= num_obs
SCIKIT_H /= num_obs

if emulator == 0:
    print("PCGP E: ", PCGP_E)
    print("PCGP H: ", PCGP_H)
elif emulator == 1:
    print("PCSK E: ", PCSK_E)
    print("PCSK H: ", PCSK_H)
elif emulator == 2:
    print("Scikit E: ", SCIKIT_E)
    print("Scikit H: ", SCIKIT_H)

Compare the emulators with LHC points with and without PCA.

In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out","7p7_pTvn_emu_out","19p6_dNdy_emu_out","19p6_pTvn_emu_out","200_dNdy_emu_out","200_pTvn_emu_out","19p6PHOBOS_dNdeta_emu_out","200PHOBOS_dNdeta_emu_out","200PHOBOS_vn_emu_out"]
foldername = 'emulator_outputs_only_LHC/emulator_output'
foldername1 = 'emulator_outputs_only_LHC/emulator_output_PCA'

emulator = 2

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([30],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

err1_list_PCA = []
err2_list_PCA = []
err3_list_PCA = []
hon1_list_PCA = []
hon2_list_PCA = []
hon3_list_PCA = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([30],foldername1,file)
    err1_list_PCA.append(err[0])
    err2_list_PCA.append(err[1])
    err3_list_PCA.append(err[2])
    hon1_list_PCA.append(hon[0])
    hon2_list_PCA.append(hon[1])
    hon3_list_PCA.append(hon[2])

light_red = (1.0, 0.6, 0.6)  # Light red
light_blue = (0.6, 0.6, 1.0)  # Light blue
light_green = (0.6, 0.9, 0.6)  # Light green
colors = ['r','b','g',light_red,light_blue,light_green]
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{7.7\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'$\langle p_{\rm T}\rangle|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{200\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          r'$v_2^{\rm ch}(\eta)|^{200\;\mathrm{GeV}}$'
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(err1_list)):  # Iterate over the remaining data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP PCA', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK PCA', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP PCA', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
if emulator == 0:
    plt.savefig("err_LHC_PCA_vs_noPCA_PCGP.pdf")
elif emulator == 1:
    plt.savefig("err_LHC_PCA_vs_noPCA_PCSK.pdf")
elif emulator == 2:
    plt.savefig("err_LHC_PCA_vs_noPCA_Scikit.pdf")

# honesty plot
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(hon1_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP PCA', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK PCA', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP PCA', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-1, 1])
axs[1].set_ylim([-1, 1])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
if emulator == 0:
    plt.savefig("hon_LHC_PCA_vs_noPCA_PCGP.pdf")
elif emulator == 1:
    plt.savefig("hon_LHC_PCA_vs_noPCA_PCSK.pdf")
elif emulator == 2:
    plt.savefig("hon_LHC_PCA_vs_noPCA_Scikit.pdf")


Compare the emulators with LHC points (no LOG, no PCA) with the ones including posterior points (LOG, PCA).

In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out","7p7_pTvn_emu_out","19p6_dNdy_emu_out","19p6_pTvn_emu_out","200_dNdy_emu_out","200_pTvn_emu_out","19p6PHOBOS_dNdeta_emu_out","200PHOBOS_dNdeta_emu_out","200PHOBOS_vn_emu_out"]
filename_prefix_list_LOG = ["7p7_LOGdNdy_emu_out","7p7_pTvn_emu_out","19p6_LOGdNdy_emu_out","19p6_pTvn_emu_out","200_LOGdNdy_emu_out","200_pTvn_emu_out","19p6PHOBOS_LOGdNdeta_emu_out","200PHOBOS_LOGdNdeta_emu_out","200PHOBOS_vn_emu_out"]
foldername = 'emulator_outputs_with_posterior/emulator_output'
foldername1 = 'emulator_outputs_with_posterior/emulator_output_PCA'

emulator = 2

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([100],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

err1_list_PCA = []
err2_list_PCA = []
err3_list_PCA = []
hon1_list_PCA = []
hon2_list_PCA = []
hon3_list_PCA = []
for file in filename_prefix_list_LOG:
    err, hon = read_multiple_emulator_errors_files([30],foldername1,file)
    err1_list_PCA.append(err[0])
    err2_list_PCA.append(err[1])
    err3_list_PCA.append(err[2])
    hon1_list_PCA.append(hon[0])
    hon2_list_PCA.append(hon[1])
    hon3_list_PCA.append(hon[2])

light_red = (1.0, 0.6, 0.6)  # Light red
light_blue = (0.6, 0.6, 1.0)  # Light blue
light_green = (0.6, 0.9, 0.6)  # Light green
colors = ['r','b','g',light_red,light_blue,light_green]
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{7.7\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'$\langle p_{\rm T}\rangle|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{200\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          r'$v_2^{\rm ch}(\eta)|^{200\;\mathrm{GeV}}$'
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(err1_list)):  # Iterate over the remaining data sets
    for obs in range(len(err1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, err1_list_PCA[dataset][-1][obs], color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, err2_list_PCA[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, err3_list[dataset][-1][obs], color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, err3_list_PCA[dataset][-1][obs], color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP LHC', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP PCA + log', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK LHC', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK PCA + log', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP LHC', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP PCA + log', linestyle='None', fillstyle='none')
    ]
axs[1].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
if emulator == 0:
    plt.savefig("err_LHC_vs_POST_logPCA_PCGP.pdf")
elif emulator == 1:
    plt.savefig("err_LHC_vs_POST_logPCA_PCSK.pdf")
elif emulator == 2:
    plt.savefig("err_LHC_vs_POST_logPCA_Scikit.pdf")

# honesty plot
fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[0].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[0].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[0].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[0].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[0].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[0].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(hon1_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon1_list[dataset][-1])):
        if emulator == 0:
            axs[1].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
            axs[1].scatter(idx_observable, np.log(hon1_list_PCA[dataset][-1][obs]), color=colors[3], marker='o', s=10, facecolors='None')
        elif emulator == 1:
            axs[1].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
            axs[1].scatter(idx_observable, np.log(hon2_list_PCA[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        elif emulator == 2:
            axs[1].scatter(idx_observable, np.log(hon3_list[dataset][-1][obs]), color=colors[2], marker='^', s=10)
            axs[1].scatter(idx_observable, np.log(hon3_list_PCA[dataset][-1][obs]), color=colors[5], marker='^', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)

if emulator == 0:
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP LHC', linestyle='None'),
        plt.Line2D([0], [0], marker='o', color=colors[3], markersize=10, label='PCGP PCA + log', linestyle='None', fillstyle='none')
    ]
elif emulator == 1:
    legend_elements = [
        plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK LHC', linestyle='None'),
        plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label='PCSK PCA + log', linestyle='None', fillstyle='none')
    ]
elif emulator == 2:
    legend_elements = [
        plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP LHC', linestyle='None'),
        plt.Line2D([0], [0], marker='^', color=colors[5], markersize=10, label='Scikit GP PCA + log', linestyle='None', fillstyle='none')
    ]
axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-2.3, 2.3])
axs[1].set_ylim([-2.3, 2.3])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
if emulator == 0:
    plt.savefig("hon_LHC_vs_POST_logPCA_PCGP.pdf")
elif emulator == 1:
    plt.savefig("hon_LHC_vs_POST_logPCA_PCSK.pdf")
elif emulator == 2:
    plt.savefig("hon_LHC_vs_POST_logPCA_Scikit.pdf")


Compare the PCSK with itself, once with the uncertainty included and once without. Then do the same comparison with the PCGP emulator to see if the emulators are similar in that case. 

In [ ]:
def read_multiple_emulator_errors_files_PCSK_noERR(number_test_points_list,filename):
    data_list2 = []
    for i in number_test_points_list:
        filename2 = f"./emulator_outputs_only_LHC/emulator_output_PCSK_0ERR/{filename}_{i}_pred_err_obs_2.dat"
        data2 = read_emulator_file_errors(filename2)
        data_list2.extend([data2])

    data_list5 = []
    for i in number_test_points_list:
        filename5 = f"./emulator_outputs_only_LHC/emulator_output_PCSK_0ERR/{filename}_{i}_GP_honesty_obs_2.dat"
        data5 = read_emulator_file_errors(filename5)
        data_list5.extend([data5])

    return data_list2, data_list5

def read_multiple_emulator_errors_files_PCSK_2ERR(number_test_points_list,filename):
    data_list2 = []
    for i in number_test_points_list:
        filename2 = f"./emulator_outputs_only_LHC/emulator_output_PCSK_2ERR/{filename}_{i}_pred_err_obs_2.dat"
        data2 = read_emulator_file_errors(filename2)
        data_list2.extend([data2])

    data_list5 = []
    for i in number_test_points_list:
        filename5 = f"./emulator_outputs_only_LHC/emulator_output_PCSK_2ERR/{filename}_{i}_GP_honesty_obs_2.dat"
        data5 = read_emulator_file_errors(filename5)
        data_list5.extend([data5])

    return data_list2, data_list5

foldername = 'emulator_outputs_only_LHC/emulator_output'
filename_prefix_list = ["7p7_dNdy_emu_out",
                        "7p7_pTvn_emu_out",
                        "19p6_dNdy_emu_out",
                        "19p6_pTvn_emu_out",
                        "200_dNdy_emu_out",
                        "200_pTvn_emu_out",
                        "19p6PHOBOS_dNdeta_emu_out",
                        "200PHOBOS_dNdeta_emu_out",
                        "200PHOBOS_vn_emu_out"]

err2_list_0ERR = []
hon2_list_0ERR = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files_PCSK_noERR([200,100,30],file)
    err2_list_0ERR.append(err)
    hon2_list_0ERR.append(hon)

err2_list = []
hon2_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([200,100,30],foldername,file)
    err2_list.append(err[1])
    hon2_list.append(hon[1])

err2_list_2ERR = []
hon2_list_2ERR = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files_PCSK_2ERR([200,100,30],file)
    err2_list_2ERR.append(err)
    hon2_list_2ERR.append(hon)

colors = ['r','b','g','magenta','teal','orange','purple','grey','seagreen']
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{7.7\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'$\langle p_{\rm T}\rangle|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{200\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          r'$v_2^{\rm ch}(\eta)|^{200\;\mathrm{GeV}}$'
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err2_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err2_list[dataset][-1])):
        axs[0].scatter(idx_observable, err2_list_0ERR[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        axs[0].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
        axs[0].scatter(idx_observable, err2_list_2ERR[dataset][-1][obs], color=colors[3], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err2_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(err2_list)):  # Iterate over the remaining data sets
    for obs in range(len(err2_list[dataset][-1])):
        axs[1].scatter(idx_observable, err2_list_0ERR[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        axs[1].scatter(idx_observable, err2_list[dataset][-1][obs], color=colors[1], marker='s', s=10)
        axs[1].scatter(idx_observable, err2_list_2ERR[dataset][-1][obs], color=colors[3], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err2_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)
legend_elements = [
    plt.Line2D([0], [0], marker='d', color=colors[4], markersize=10, label=r'PCSK $0\sigma$', linestyle='None', fillstyle='none'),
    plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label=r'PCSK $1\sigma$', linestyle='None'),
    plt.Line2D([0], [0], marker='s', color=colors[3], markersize=10, label=r'PCSK $2\sigma$', linestyle='None', fillstyle='none')
]

axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
plt.savefig("err_full_training_set_PCSK_ERR_noPCA.pdf")

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon2_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon2_list[dataset][-1])):
        axs[0].scatter(idx_observable, np.log(hon2_list_0ERR[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        axs[0].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
        axs[0].scatter(idx_observable, np.log(hon2_list_2ERR[dataset][-1][obs]), color=colors[3], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon2_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(hon2_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon2_list[dataset][-1])):
        axs[1].scatter(idx_observable, np.log(hon2_list_0ERR[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        axs[1].scatter(idx_observable, np.log(hon2_list[dataset][-1][obs]), color=colors[1], marker='s', s=10)
        axs[1].scatter(idx_observable, np.log(hon2_list_2ERR[dataset][-1][obs]), color=colors[3], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon2_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)
legend_elements = [
    plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label=r'PCSK $0\sigma$', linestyle='None', fillstyle='none'),
    plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label=r'PCSK $1\sigma$', linestyle='None'),
    plt.Line2D([0], [0], marker='s', color=colors[3], markersize=10, label=r'PCSK $2\sigma$', linestyle='None', fillstyle='none')
]

axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.05, borderaxespad=0.15)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-1, 1])
axs[1].set_ylim([-1, 1])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
plt.savefig("hon_full_training_set_PCSK_ERR_noPCA.pdf")


PCSK no uncertainty and PCGP comparison:

In [ ]:
def read_multiple_emulator_errors_files_PCSKERR(number_test_points_list,filename):
    data_list2 = []
    for i in number_test_points_list:
        filename2 = f"./emulator_outputs_only_LHC/emulator_output_PCSK_0ERR/{filename}_{i}_pred_err_obs_2.dat"
        data2 = read_emulator_file_errors(filename2)
        data_list2.extend([data2])

    data_list5 = []
    for i in number_test_points_list:
        filename5 = f"./emulator_outputs_only_LHC/emulator_output_PCSK_0ERR/{filename}_{i}_GP_honesty_obs_2.dat"
        data5 = read_emulator_file_errors(filename5)
        data_list5.extend([data5])

    return data_list2, data_list5

foldername = 'emulator_outputs_only_LHC/emulator_output'
filename_prefix_list = ["7p7_dNdy_emu_out",
                        "7p7_pTvn_emu_out",
                        "19p6_dNdy_emu_out",
                        "19p6_pTvn_emu_out",
                        "200_dNdy_emu_out",
                        "200_pTvn_emu_out",
                        "19p6PHOBOS_dNdeta_emu_out",
                        "200PHOBOS_dNdeta_emu_out",
                        "200PHOBOS_vn_emu_out"]

err1_list = []
hon1_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([200,100,30],foldername,file)
    err1_list.append(err[0])
    hon1_list.append(hon[0])

err2_list_0ERR = []
hon2_list_0ERR = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files_PCSKERR([200,100,30],file)
    err2_list_0ERR.append(err)
    hon2_list_0ERR.append(hon)

colors = ['r','b','g','magenta','teal','orange','purple','grey','seagreen']
labels = [r'${\rm d}N/{\rm d}y|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{7.7\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p}$',
          r'$\langle p_{\rm T}\rangle|^{19.6\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N/{\rm d}y|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$',
          r'$\langle p_{\rm T}\rangle|^{200\;\mathrm{GeV}}_{\pi^+,K^+,p,\bar{p}}$' + '\\\n' + r'$v_{2/3}^{\rm ch}\lbrace 2\rbrace|^{200\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{19.6\;\mathrm{GeV}}$',
          r'${\rm d}N_{\rm ch}/{\rm d}\eta|^{200\;\mathrm{GeV}}$',
          r'$v_2^{\rm ch}(\eta)|^{200\;\mathrm{GeV}}$'
          ]

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(err1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(err1_list[dataset][-1])):
        axs[0].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
        axs[0].scatter(idx_observable, err2_list_0ERR[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(err1_list)):  # Iterate over the remaining data sets
    for obs in range(len(err1_list[dataset][-1])):
        axs[1].scatter(idx_observable, err1_list[dataset][-1][obs], color=colors[0], marker='o', s=10)
        axs[1].scatter(idx_observable, err2_list_0ERR[dataset][-1][obs], color=colors[4], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(err1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)
legend_elements = [
    plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
    plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label=r'PCSK $0\sigma$', linestyle='None', fillstyle='none')
]

axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{E}$")
axs[1].set_ylabel(r"$\mathcal{E}$")
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[0].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
axs[1].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
axs[1].yaxis.set_major_locator(ticker.LogLocator(subs=[1.0, 2.0, 5.0]))  # Ensure at least two labels
plt.tight_layout()
plt.savefig("err_full_training_set_PCSK_0ERR_PCGP_noPCA.pdf")

fig, axs = plt.subplots(2, 1, figsize=(12, 6))
plt.rc('font', size=18)
idx_dataset = 0
idx_observable = 0
dataset_boundaries = [0]
for dataset in range(len(hon1_list[:6])):  # Only iterate over the first 5 data sets
    for obs in range(len(hon1_list[dataset][-1])):
        axs[0].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
        axs[0].scatter(idx_observable, np.log(hon2_list_0ERR[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[:6]):
        axs[0].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))  

# Plot for the bottom part
idx_dataset = 0
idx_observable = 0
dataset_boundaries1 = [0]
for dataset in range(6, len(hon1_list)):  # Iterate over the remaining data sets
    for obs in range(len(hon1_list[dataset][-1])):
        axs[1].scatter(idx_observable, np.log(hon1_list[dataset][-1][obs]), color=colors[0], marker='o', s=10)
        axs[1].scatter(idx_observable, np.log(hon2_list_0ERR[dataset][-1][obs]), color=colors[4], marker='s', s=10, facecolors='None')
        idx_observable += 1

    dataset_boundaries1.append(idx_observable)
    idx_dataset += 1
    if idx_dataset < len(hon1_list[6:]):
        axs[1].axvline(x=idx_observable, color='k', linestyle='--', zorder=(-10))

# Compute the average index for each dataset
average_indices1 = [(start + end) // 2 for start, end in zip(dataset_boundaries[:-1], dataset_boundaries[1:])]
average_indices2 = [(start + end) // 2 for start, end in zip(dataset_boundaries1[:-1], dataset_boundaries1[1:])]

# Set the x-labels at the average indices
axs[0].set_xticks(average_indices1)
axs[1].set_xticks(average_indices2)
axs[0].set_xticklabels(labels[:6], rotation=0, ha='center', fontsize=15)
axs[1].set_xticklabels(labels[6:], rotation=0, ha='center', fontsize=15)
legend_elements = [
    plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
    plt.Line2D([0], [0], marker='s', color=colors[4], markersize=10, label=r'PCSK $0\sigma$', linestyle='None', fillstyle='none')
]

axs[0].legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25)
axs[0].set_xlim([-1, dataset_boundaries[-1] + 1])
axs[1].set_xlim([-1, dataset_boundaries1[-1] + 1])
axs[0].set_ylabel(r"$\mathcal{H}$")
axs[1].set_ylabel(r"$\mathcal{H}$")
axs[0].set_ylim([-1, 1])
axs[1].set_ylim([-1, 1])
axs[0].axhline(y=0, color='k', linestyle=':', zorder=(-10))
axs[1].axhline(y=0, color='k', linestyle=':', zorder=(-10))
plt.tight_layout()
plt.savefig("hon_full_training_set_PCSK_0ERR_PCGP_noPCA.pdf")

Quick summary:
- The AuAu7.7_dNdy data set does not sort out any of the 1100 training points.
- The AuAu7.7_pTvn data set sorts out 40 of the 1100 training points due to large statistical errors.
- The AuAu19p6_dNdy data set does not sort out any of the 1100 training points.
- The AuAu19p6_pTvn data set sorts out 5 of the 1100 training points due to large statistical errors.
- The AuAu19p6_PHOBOS_dNdeta data set does not sort out any of the 1100 points.
- The AuAu200_dNdy data set does not sort out any of the 1100 training points.
- The AuAu200_pTvn data set sorts out 46 of the 1100 training points due to large statistical errors.
- The AuAu200_PHOBOS_dNdeta data set does not sort out any of the 1100 training points.
- The AuAu200_PHOBOSv2eta data set does not sort out any of the 1100 training points.

In [ ]:
filename_prefix_list = ["7p7_dNdy_emu_out",
                        "7p7_pTvn_emu_out",
                        "19p6_dNdy_emu_out",
                        "19p6_pTvn_emu_out",
                        "200_dNdy_emu_out",
                        "200_pTvn_emu_out",
                        "19p6PHOBOS_dNdeta_emu_out",
                        "200PHOBOS_dNdeta_emu_out",
                        "200PHOBOS_vn_emu_out"]
filename_prefix_list_LOG = ["7p7_LOGdNdy_emu_out",
                            "19p6_LOGdNdy_emu_out",
                            "200_LOGdNdy_emu_out",
                            "19p6PHOBOS_LOGdNdeta_emu_out",
                            "200PHOBOS_LOGdNdeta_emu_out"]
datasets_LOG = [0,2,4,6,7]
colors = ['r','b','g','magenta','teal','orange','purple','grey','seagreen']
foldername = 'emulator_outputs_only_LHC/emulator_output'

annotation_list_parameters = [
    r'$\langle p_{\mathrm{T}}\rangle|^{7.7\;\mathrm{GeV}}_{\pi^+}$',
    r'$\mathrm{d}N/\mathrm{d}y|^{200\;\mathrm{GeV}}_{\pi^+}$',
    r'$v_2^{\mathrm{ch}}(\eta)|^{200\;\mathrm{GeV}}$'
]

err1_list = []
err2_list = []
err3_list = []
hon1_list = []
hon2_list = []
hon3_list = []
for file in filename_prefix_list:
    err, hon = read_multiple_emulator_errors_files([900,800,700,600,500,400,300,200,100,30],foldername,file)
    err1_list.append(err[0])
    err2_list.append(err[1])
    err3_list.append(err[2])
    hon1_list.append(hon[0])
    hon2_list.append(hon[1])
    hon3_list.append(hon[2])

err1_list_PCA = []
err2_list_PCA = []
err3_list_PCA = []
hon1_list_PCA = []
hon2_list_PCA = []
hon3_list_PCA = []
for file in filename_prefix_list_LOG:
    err, hon = read_multiple_emulator_errors_files([900,800,700,600,500,400,300,200,100,30],foldername,file)
    err1_list_PCA.append(err[0])
    err2_list_PCA.append(err[1])
    err3_list_PCA.append(err[2])
    hon1_list_PCA.append(hon[0])
    hon2_list_PCA.append(hon[1])
    hon3_list_PCA.append(hon[2])

numberValidTrainingPoints_withPosterior = [
    [200,300,400,500,600,700,800,900,1000,1070],
    [160,260,360,460,560,660,760,860,960,1030],
    [200,300,400,500,600,700,800,900,1000,1070],
    [195,295,395,495,595,695,795,895,995,1065],
    [200,300,400,500,600,700,800,900,1000,1070],
    [200,300,400,500,600,700,800,900,1000,1070],
    [154,254,354,454,554,654,754,854,954,1024],
    [200,300,400,500,600,700,800,900,1000,1070],
    [200,300,400,500,600,700,800,900,1000,1070]
]

numberValidTrainingPoints = [
    [100,200,300,400,500,600,700,800,900,970],
    [60,160,260,360,460,560,660,760,860,930],
    [100,200,300,400,500,600,700,800,900,970],
    [95,195,295,395,495,595,695,795,895,965],
    [100,200,300,400,500,600,700,800,900,970],
    [100,200,300,400,500,600,700,800,900,970],
    [54,154,254,354,454,554,654,754,854,924],
    [100,200,300,400,500,600,700,800,900,970],
    [100,200,300,400,500,600,700,800,900,970]
]

idx_dataset_LOG = 0
for d in range(len(filename_prefix_list)):
    numTrain = numberValidTrainingPoints[d]
    numObs = len(err1_list[d][0])
    for o in range(numObs):
        fig, axs = plt.subplots(1, 1, figsize=(6, 4))
        plt.rc('font', size=15)
        for i in range(len(numTrain)):
            axs.scatter(numTrain[i], err1_list[d][i][o], color=colors[0], marker='o', s=23)
            axs.scatter(numTrain[i], err2_list[d][i][o], color=colors[1], marker='s', s=23)
            axs.scatter(numTrain[i], err3_list[d][i][o], color=colors[2], marker='^', s=23)
            #if d in datasets_LOG:
            #    axs.scatter(numTrain[i], err1_list_LOG[idx_dataset_LOG][i][o], color=colors[0], marker='o', s=14, facecolors='None')
            #    axs.scatter(numTrain[i], err2_list_LOG[idx_dataset_LOG][i][o], color=colors[1], marker='s', s=14, facecolors='None')
            #    axs.scatter(numTrain[i], err3_list_LOG[idx_dataset_LOG][i][o], color=colors[2], marker='^', s=14, facecolors='None')
        legend_elements = [
            plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
            plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
            plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None')
        ]
        axs.legend(handles=legend_elements, loc='upper right', ncols=1, borderpad=0.12, borderaxespad=0.25, fontsize=12)
        axs.set_ylabel(r"$\mathcal{E}$")
        axs.set_xlabel(r"training points")
        axs.set_xscale('log')
        axs.set_yscale('log')
        axs.yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
        axs.yaxis.set_minor_formatter(StrMethodFormatter('{x:.2f}'))

        if d == 1 and o == 0:
            axs.annotate(annotation_list_parameters[0], xy=(0.2, 0.2), xycoords='axes fraction', fontsize=15, ha='center', va='center')
        elif d == 4 and o == 0:
            axs.annotate(annotation_list_parameters[1], xy=(0.2, 0.2), xycoords='axes fraction', fontsize=15, ha='center', va='center')
        elif d == 8:
            axs.annotate(annotation_list_parameters[2], xy=(0.2, 0.2), xycoords='axes fraction', fontsize=15, ha='center', va='center')
        plt.tight_layout()
        plt.savefig(f"./plots/err_training_points_PCA/err_dataset{d}_obs{o}_PCA.pdf")
    if d in datasets_LOG:
        idx_dataset_LOG += 1

idx_dataset_LOG = 0
for d in range(len(filename_prefix_list)):
    numTrain = numberValidTrainingPoints[d]
    numObs = len(err1_list[d][0])
    for o in range(numObs):
        fig, axs = plt.subplots(1, 1, figsize=(6, 4))
        plt.rc('font', size=15)
        for i in range(len(numTrain)):
            axs.scatter(numTrain[i], np.log(hon1_list[d][i][o]), color=colors[0], marker='o', s=23)
            axs.scatter(numTrain[i], np.log(hon2_list[d][i][o]), color=colors[1], marker='s', s=23)
            axs.scatter(numTrain[i], np.log(hon3_list[d][i][o]), color=colors[2], marker='^', s=23)
            #if d in datasets_LOG:
            #    axs.scatter(numTrain[i], hon1_list_LOG[idx_dataset_LOG][i][o], color=colors[0], marker='o', s=14, facecolors='None')
            #    axs.scatter(numTrain[i], hon2_list_LOG[idx_dataset_LOG][i][o], color=colors[1], marker='s', s=14, facecolors='None')
            #    axs.scatter(numTrain[i], hon3_list_LOG[idx_dataset_LOG][i][o], color=colors[2], marker='^', s=14, facecolors='None')
        legend_elements = [
            plt.Line2D([0], [0], marker='o', color=colors[0], markersize=10, label='PCGP', linestyle='None'),
            plt.Line2D([0], [0], marker='s', color=colors[1], markersize=10, label='PCSK', linestyle='None'),
            plt.Line2D([0], [0], marker='^', color=colors[2], markersize=10, label='Scikit GP', linestyle='None')
        ]
        axs.legend(handles=legend_elements, loc='upper right', ncols=3, borderpad=0.12, borderaxespad=0.25, fontsize=12)
        axs.set_ylabel(r"$\mathcal{H}$")
        axs.set_xlabel(r"training points")
        axs.set_xscale('log')
        axs.set_ylim([-1, 1])
        axs.axhline(y=0, color='k', linestyle=':', zorder=(-10))
        if d == 1 and o == 0:
            axs.annotate(annotation_list_parameters[0], xy=(0.2, 0.2), xycoords='axes fraction', fontsize=15, ha='center', va='center')
        elif d == 4 and o == 0:
            axs.annotate(annotation_list_parameters[1], xy=(0.2, 0.2), xycoords='axes fraction', fontsize=15, ha='center', va='center')
        elif d == 8:
            axs.annotate(annotation_list_parameters[2], xy=(0.2, 0.2), xycoords='axes fraction', fontsize=15, ha='center', va='center')
        plt.tight_layout()
        plt.savefig(f"./plots/hon_log/hon_training_points_noPCA/hon_dataset{d}_obs{o}_noPCA.pdf")
    if d in datasets_LOG:
        idx_dataset_LOG += 1